# Module 1: The Simulation Engine 🏭

**Author**: Your Name  
**Project**: CausalCommerce - Business Strategy Simulator  
**Goal**: Demonstrate understanding of causal inference through synthetic data generation

---

## What Makes This Special?

Most portfolio projects use **static datasets** from Kaggle. This project builds a **dynamic simulation** that:

1. **Generates realistic customer populations** with hidden causal structures
2. **Simulates business interventions** (discounts, ads, etc.)
3. **Introduces realistic bias** that breaks naive analysis
4. **Provides ground truth** so we can validate our causal inference methods

This approach demonstrates **deep understanding** of:
- Causal DAGs (Directed Acyclic Graphs)
- Selection bias and confounding
- Treatment effect heterogeneity
- Real business constraints

---

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from data.generators import (
    SimulationConfig,
    generate_customer_data,
    BehaviorSimulator,
    simulate_scenario,
    SEGMENT_PARAMS
)

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("✅ Setup complete!")

---

## 2. Understanding the Customer Segments

Our simulation models 4 distinct customer types (inspired by real uplift modeling research):

| Segment | Base Purchase Rate | Discount Sensitivity | Key Insight |
|---------|-------------------|---------------------|-------------|
| **Loyalists** | 80% | +5% | Buy anyway - discount wastes money |
| **Persuadables** | 30% | +40% | **TARGET THESE!** High ROI |
| **Sleeping Dogs** | 15% | -10% | Discounts HURT conversion |
| **Lost Causes** | 5% | +2% | Won't buy even with discount |

**The Challenge**: In real business, we can't observe segments directly. We only see:
- Observable features (age, activity, past purchases)
- Outcomes (did they buy?)

**Goal**: Use causal inference to identify Persuadables without wasting money on others.

In [ ]:
# Visualize segment parameters
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Purchase probabilities
segments = list(SEGMENT_PARAMS.keys())
base_probs = [SEGMENT_PARAMS[s].base_purchase_prob for s in segments]
discount_effects = [SEGMENT_PARAMS[s].discount_sensitivity for s in segments]

x = np.arange(len(segments))
width = 0.35

axes[0].bar(x - width/2, base_probs, width, label='Base Purchase Prob', alpha=0.8)
axes[0].bar(x + width/2, discount_effects, width, label='Discount Effect', alpha=0.8)
axes[0].set_xlabel('Customer Segment')
axes[0].set_ylabel('Probability')
axes[0].set_title('Purchase Behavior by Segment')
axes[0].set_xticks(x)
axes[0].set_xticklabels(segments, rotation=45)
axes[0].legend()
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Plot 2: Expected lift from 20% discount
discount_amount = 0.20
expected_lifts = [SEGMENT_PARAMS[s].discount_sensitivity * discount_amount * 100 for s in segments]
colors = ['green' if lift > 0 else 'red' for lift in expected_lifts]

axes[1].bar(segments, expected_lifts, color=colors, alpha=0.7)
axes[1].set_xlabel('Customer Segment')
axes[1].set_ylabel('Expected Lift (%)')
axes[1].set_title('Expected Conversion Lift from 20% Discount')
axes[1].set_xticklabels(segments, rotation=45)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: 'Sleeping Dogs' have NEGATIVE lift - discounts hurt conversion!")
print("   This is realistic: some customers see discounts as 'cheap' or spammy.")

---

## 3. Generate Customer Population

Let's create a synthetic population of 10,000 customers.

In [ ]:
# Configure simulation
config = SimulationConfig(
    n_customers=10000,
    random_seed=42
)

# Generate customers
customers = generate_customer_data(config)

print(f"Generated {len(customers):,} customers\n")
print("Segment distribution:")
print(customers['segment'].value_counts(normalize=True).sort_values(ascending=False))
print("\nFirst few customers:")
customers.head()

### 3.1 Examine Observable Features

These are features we CAN measure in real business (like from a CRM system).

In [ ]:
observable_features = [
    'age', 'tenure_months', 'activity_score',
    'prev_purchases', 'account_value', 'email_engagement_rate'
]

print("Observable Features Summary:\n")
print(customers[observable_features].describe())

# Visualize feature distributions by segment
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, feature in enumerate(observable_features):
    for segment in customers['segment'].unique():
        data = customers[customers['segment'] == segment][feature]
        axes[i].hist(data, alpha=0.5, label=segment, bins=30)
    axes[i].set_title(feature)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')
    if i == 0:
        axes[i].legend()

plt.tight_layout()
plt.show()

print("\n💡 Notice: Features are CORRELATED with segment but overlapping.")
print("   This is realistic - we can't perfectly identify segments from observables.")

### 3.2 Examine Hidden Propensities (Ground Truth)

In real life, we NEVER observe these. But in our simulation, we know them!

This allows us to:
1. Validate our causal inference methods
2. Calculate the TRUE treatment effect
3. Measure how much money naive analysis wastes

In [ ]:
# Compare true propensities by segment
propensity_cols = [
    'base_purchase_propensity',
    'discount_effect',
    'churn_propensity',
    'avg_purchase_value'
]

print("Ground Truth: True Propensities by Segment\n")
ground_truth = customers.groupby('segment')[propensity_cols].mean()
print(ground_truth.round(3))

# Visualize the key insight: discount_effect
plt.figure(figsize=(10, 6))
sns.boxplot(data=customers, x='segment', y='discount_effect')
plt.title('Distribution of TRUE Discount Effects by Segment', fontsize=14, fontweight='bold')
plt.xlabel('Customer Segment')
plt.ylabel('Discount Effect (% increase in purchase probability)')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Zero effect')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n🎯 This is what we're trying to ESTIMATE with causal inference!")
print("   Challenge: We only observe outcomes, not these propensities.")

---

## 4. Simulate Business Scenarios

Now let's simulate what happens when we run marketing campaigns.

### 4.1 Scenario A: Perfect Randomized A/B Test

This is the **gold standard** - random treatment assignment with no bias.

In [ ]:
# Simulate randomized experiment
simulator = BehaviorSimulator(customers, config)
experiment_random = simulator.simulate_experiment(
    treatment_assignment='random',
    treatment_probability=0.5,
    discount_amount=0.20
)

print("Randomized A/B Test Results")
print("="*60)

# Check balance
print("\nTreatment Balance:")
print(experiment_random['treated'].value_counts())

# Naive analysis (what most people do)
print("\nNaive Comparison (Simple Average):")
naive_results = experiment_random.groupby('treated').agg({
    'purchased': ['mean', 'sum'],
    'revenue': 'mean',
    'churned': 'mean'
})
print(naive_results)

control_conv = naive_results.loc[0, ('purchased', 'mean')]
treated_conv = naive_results.loc[1, ('purchased', 'mean')]
naive_lift = (treated_conv - control_conv) / control_conv * 100

print(f"\n📊 Naive Lift: {naive_lift:.2f}%")
print("\n✅ In a randomized experiment, naive analysis is CORRECT!")
print("   No confounding, so simple comparison gives true causal effect.")

### 4.2 Scenario B: Biased Targeting (The Real World)

In reality, treatments are NOT randomly assigned. Marketing teams target "engaged" customers.

**This creates confounding**: Active customers buy more anyway, even without treatment!

In [ ]:
# Simulate biased targeting
experiment_biased = simulator.simulate_experiment(
    treatment_assignment='biased_activity',
    discount_amount=0.20
)

print("Biased Targeting Results (Marketing targets active customers)")
print("="*60)

# Show the bias in treatment assignment
print("\nTreatment Assignment by Activity Level:")
activity_bins = pd.cut(experiment_biased['activity_score'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
treatment_by_activity = experiment_biased.groupby(activity_bins)['treated'].agg(['mean', 'count'])
treatment_by_activity.columns = ['Treatment Rate', 'Count']
print(treatment_by_activity)

print("\n⚠️  BIAS DETECTED: Higher activity customers get more treatment!")

# Naive analysis (BIASED!)
print("\nNaive Comparison:")
naive_biased = experiment_biased.groupby('treated').agg({
    'purchased': ['mean', 'sum'],
    'revenue': 'mean'
})
print(naive_biased)

control_conv_b = naive_biased.loc[0, ('purchased', 'mean')]
treated_conv_b = naive_biased.loc[1, ('purchased', 'mean')]
naive_lift_biased = (treated_conv_b - control_conv_b) / control_conv_b * 100

print(f"\n📊 Naive Lift: {naive_lift_biased:.2f}%")
print(f"📊 True Lift (from random experiment): {naive_lift:.2f}%")
print(f"\n⚠️  OVERESTIMATE: {naive_lift_biased - naive_lift:.2f} percentage points!")
print("\n🚨 This is why we need causal inference!")
print("   The naive analysis attributes natural differences to the treatment.")

### 4.3 Visualize the Confounding Problem

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Activity score distribution by treatment
axes[0].hist(experiment_biased[experiment_biased['treated']==0]['activity_score'], 
             alpha=0.6, bins=30, label='Control', color='blue')
axes[0].hist(experiment_biased[experiment_biased['treated']==1]['activity_score'], 
             alpha=0.6, bins=30, label='Treated', color='orange')
axes[0].set_xlabel('Activity Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Confounding: Activity Score by Treatment Group')
axes[0].legend()

# Plot 2: Purchase rate by activity and treatment
activity_deciles = pd.qcut(experiment_biased['activity_score'], q=10, duplicates='drop')
purchase_by_activity = experiment_biased.groupby([activity_deciles, 'treated'])['purchased'].mean().unstack()
purchase_by_activity.plot(ax=axes[1], marker='o')
axes[1].set_xlabel('Activity Decile')
axes[1].set_ylabel('Purchase Rate')
axes[1].set_title('Purchase Rate by Activity Level and Treatment')
axes[1].legend(['Control', 'Treated'])
axes[1].set_xticklabels(range(1, len(purchase_by_activity)+1))

plt.tight_layout()
plt.show()

print("\n💡 Notice: The treatment and control groups have DIFFERENT activity distributions.")
print("   This confounds our analysis - we're comparing apples to oranges!")

---

## 5. Calculate TRUE Treatment Effects (Ground Truth)

Since we generated the data, we can calculate the TRUE causal effect!

This will be our benchmark when we apply causal inference methods in Module 3.

In [ ]:
# Calculate true Individual Treatment Effect (ITE) for each customer
discount_amount = 0.20
experiment_biased['true_ITE'] = experiment_biased['discount_effect'] * discount_amount

# Average Treatment Effect (ATE)
true_ATE = experiment_biased['true_ITE'].mean()

# Conditional Average Treatment Effect (CATE) by segment
true_CATE = experiment_biased.groupby('segment')['true_ITE'].mean().sort_values(ascending=False)

print("Ground Truth: TRUE Causal Effects")
print("="*60)
print(f"\nAverage Treatment Effect (ATE): {true_ATE*100:.2f}% increase in purchase probability")
print(f"\nConditional Average Treatment Effects (CATE) by Segment:")
for segment, effect in true_CATE.items():
    print(f"  {segment:20s}: {effect*100:+6.2f}%")

# Visualize
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in true_CATE.values]
plt.barh(true_CATE.index, true_CATE.values * 100, color=colors, alpha=0.7)
plt.xlabel('Treatment Effect (% increase in purchase probability)')
plt.title('TRUE Treatment Effects by Customer Segment', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=2)
plt.axvline(x=true_ATE*100, color='blue', linestyle='--', linewidth=2, label=f'ATE: {true_ATE*100:.2f}%')
plt.legend()
plt.tight_layout()
plt.show()

print("\n🎯 MODULE 3 GOAL: Estimate these effects from observed data only!")

---

## 6. Business Impact Analysis

Let's quantify the COST of getting causality wrong.

In [ ]:
# Assume we have budget to target 50% of customers
budget_pct = 0.50
n_to_target = int(len(experiment_biased) * budget_pct)
cost_per_customer = 10  # $10 cost to send discount

# Strategy 1: Random targeting (naive)
random_targets = experiment_biased.sample(n=n_to_target, random_state=42)
random_conversions = (random_targets['base_purchase_propensity'] + 
                     random_targets['true_ITE']).sum()
random_cost = n_to_target * cost_per_customer
random_roi = (random_conversions * 100 - random_cost) / random_cost  # Assume $100 per purchase

# Strategy 2: Target by observable activity (common practice)
activity_targets = experiment_biased.nlargest(n_to_target, 'activity_score')
activity_conversions = (activity_targets['base_purchase_propensity'] + 
                       activity_targets['true_ITE']).sum()
activity_cost = n_to_target * cost_per_customer
activity_roi = (activity_conversions * 100 - activity_cost) / activity_cost

# Strategy 3: OPTIMAL - Target by true treatment effect (what causal inference aims for)
optimal_targets = experiment_biased.nlargest(n_to_target, 'true_ITE')
optimal_conversions = (optimal_targets['base_purchase_propensity'] + 
                      optimal_targets['true_ITE']).sum()
optimal_cost = n_to_target * cost_per_customer
optimal_roi = (optimal_conversions * 100 - optimal_cost) / optimal_cost

# Summary
print("Business Impact: ROI Comparison")
print("="*60)
print(f"Budget: ${n_to_target * cost_per_customer:,} (target {n_to_target:,} customers)\n")

results_df = pd.DataFrame({
    'Strategy': ['Random', 'Target Active (Naive)', 'Target by ITE (Optimal)'],
    'Conversions': [random_conversions, activity_conversions, optimal_conversions],
    'ROI': [random_roi, activity_roi, optimal_roi]
})
results_df['ROI_pct'] = results_df['ROI'] * 100
print(results_df.to_string(index=False))

print(f"\n💰 Value of Causal Inference:")
print(f"   Optimal vs Random: {(optimal_roi - random_roi)*100:.1f} percentage points better ROI")
print(f"   Optimal vs Naive:  {(optimal_roi - activity_roi)*100:.1f} percentage points better ROI")
print(f"\n🎯 This is what we're building toward in Modules 3-4!")

---

## 7. Key Takeaways

### What We Built:
1. ✅ Realistic customer generator with heterogeneous treatment effects
2. ✅ Behavior simulator that creates observational data with confounding
3. ✅ Ground truth labels to validate causal inference methods

### Why This Matters for Hiring Managers:
1. **Demonstrates deep understanding** of causal inference concepts
2. **Shows engineering skills** - clean, modular, production-quality code
3. **Business-oriented** - framed in terms of ROI and strategy
4. **Innovative approach** - goes beyond static Kaggle datasets

### Next Steps:
- **Module 2**: Experimentation framework (A/B tests, observational studies)
- **Module 3**: Causal inference methods (Propensity Score, Double ML, Uplift)
- **Module 4**: Business intelligence dashboard and ROI calculator

---

## 8. Save Data for Next Modules

In [ ]:
# Save datasets for use in future modules
experiment_random.to_csv('../data/experiment_randomized.csv', index=False)
experiment_biased.to_csv('../data/experiment_biased.csv', index=False)
customers.to_csv('../data/customer_population.csv', index=False)

print("✅ Datasets saved!")
print("\nReady to move on to Module 2: Experimentation Framework")